In [1]:
# импортируем библиотеки pandas и numpy
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

In [2]:
# создаем набор из двух примеров
toy_corpus = ['Машина едет по дороге', 
              'Грузовик едет по шоссе']

In [3]:
# импортируем класс CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer
# создаем экземпляр класса CountVectorizer
vect = CountVectorizer()
# обучаем CountVectorizer
vect.fit(toy_corpus);

In [4]:
# смотрим размер и содержимое словаря
print("Размер словаря: {}".format(len(vect.vocabulary_)))
print("Содержимое словаря:\n {}".format(vect.vocabulary_))

Размер словаря: 6
Содержимое словаря:
 {'машина': 3, 'едет': 2, 'по': 4, 'дороге': 1, 'грузовик': 0, 'шоссе': 5}


In [5]:
# вычислим количество признаков 
# и посмотрим список признаков
feature_names = vect.get_feature_names_out()
print("Количество признаков: {}".format(len(feature_names)))
print("Признаки:", feature_names)

Количество признаков: 6
Признаки: ['грузовик' 'дороге' 'едет' 'машина' 'по' 'шоссе']


In [6]:
# получаем представление "мешок слов"
bag_of_words = vect.transform(toy_corpus)
print("bag_of_words: {}".format(repr(bag_of_words)))

bag_of_words: <2x6 sparse matrix of type '<class 'numpy.int64'>'
	with 8 stored elements in Compressed Sparse Row format>


In [7]:
# получаем плотное представление "мешок слов"
print("Плотное представление bag_of_words:\n{}".format(
    bag_of_words.toarray()))

Плотное представление bag_of_words:
[[0 1 1 1 1 0]
 [1 0 1 0 1 1]]


In [8]:
# создаем экземпляр класса CountVectorizer, будем 
# извлекать слова, которые встретились не менее, 
# чем в двух документах
vect = CountVectorizer(min_df=2)
# обучаем CountVectorizer
vect.fit(toy_corpus)
# смотрим содержимое словаря
print("Содержимое словаря:\n {}".format(vect.vocabulary_))

Содержимое словаря:
 {'едет': 0, 'по': 1}


In [9]:
# создаем экземпляр класса CountVectorizer, будем 
# извлекать слова, которые встретились не более, 
# чем в одном документе 
vect = CountVectorizer(max_df=1)
# обучаем CountVectorizer
vect.fit(toy_corpus)
# смотрим содержимое словаря
print("Содержимое словаря:\n {}".format(vect.vocabulary_))

Содержимое словаря:
 {'машина': 2, 'дороге': 1, 'грузовик': 0, 'шоссе': 3}


In [10]:
# создаем экземпляр класса CountVectorizer, 
# будем извлекать слова, кроме предлога по
vect = CountVectorizer(stop_words=['по'])
# обучаем CountVectorizer
vect.fit(toy_corpus)
# смотрим содержимое словаря
print("Содержимое словаря:\n {}".format(vect.vocabulary_))

Содержимое словаря:
 {'машина': 3, 'едет': 2, 'дороге': 1, 'грузовик': 0, 'шоссе': 4}


In [11]:
# создаем набор из трех примеров
toy_corpus = ['Машина едет по дороге', 
              'Машина столкнулась на дороге с автобусом', 
              'Машина сбила на дороге пешехода']
# создаем экземпляр класса CountVectorizer, будем
# извлекать три самых часто встречающихся слова
vect = CountVectorizer(max_features=3)
# обучаем CountVectorizer
vect.fit(toy_corpus)
# смотрим содержимое словаря
print("Содержимое словаря:\n {}".format(vect.vocabulary_))

Содержимое словаря:
 {'машина': 1, 'дороге': 0, 'на': 2}


In [12]:
# создаем новый набор
toy_corpus = ['Серебристая машина едет по разбитой дороге', 
              'Грязный грузовик мчится по широкому шоссе']
# создаем экземпляр класса CountVectorizer, будем извлекать
# униграммы, биграммы и триграммы
vect = CountVectorizer(ngram_range=(1, 3))
vect.fit(toy_corpus)
print("Признаки:", vect.get_feature_names_out())

Признаки: ['грузовик' 'грузовик мчится' 'грузовик мчится по' 'грязный'
 'грязный грузовик' 'грязный грузовик мчится' 'дороге' 'едет' 'едет по'
 'едет по разбитой' 'машина' 'машина едет' 'машина едет по' 'мчится'
 'мчится по' 'мчится по широкому' 'по' 'по разбитой' 'по разбитой дороге'
 'по широкому' 'по широкому шоссе' 'разбитой' 'разбитой дороге'
 'серебристая' 'серебристая машина' 'серебристая машина едет' 'широкому'
 'широкому шоссе' 'шоссе']


In [13]:
# загружаем данные
data = pd.read_csv('Data/Restaurant_Reviews.tsv', 
                   sep='\t', 
                   quoting=3)
data.head()

,Review,Liked
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [14]:
# разбиваем данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(
    data['Review'],
    data['Liked'], 
    test_size=0.3,
    stratify=data['Liked'],
    random_state=42)

In [15]:
# снова создаем экземпляр класса CountVectorizer
vect = CountVectorizer()
# обучаем CountVectorizer
vect.fit(X_train)
# вычислим количество признаков и посмотрим 
# список первых 100 признаков
feature_names = vect.get_feature_names_out()
print("Количество признаков: {}".format(
    len(feature_names)))
print("Первые 100 признаков:\n{}".format(
    feature_names[:100]))

Количество признаков: 1640
Первые 100 признаков:
['00' '10' '100' '11' '12' '15' '17' '1979' '20' '2007' '23' '30' '35'
 '40' '45' '4ths' '5lb' '70' '99' 'about' 'above' 'absolute' 'absolutely'
 'absolutley' 'accordingly' 'accountant' 'ache' 'acknowledged' 'across'
 'actual' 'actually' 'added' 'affordable' 'after' 'afternoon' 'again'
 'ago' 'airline' 'airport' 'ala' 'albondigas' 'all' 'allergy' 'almost'
 'alone' 'also' 'although' 'always' 'am' 'amazing' 'ambiance' 'ambience'
 'amount' 'an' 'and' 'andddd' 'angry' 'annoying' 'another' 'anticipated'
 'any' 'anymore' 'anyone' 'anytime' 'anyways' 'apart' 'apology' 'app'
 'appalling' 'apparently' 'appealing' 'appetizer' 'apple' 'are' 'area'
 'aren' 'aria' 'around' 'array' 'arrived' 'article' 'as' 'ask' 'asked'
 'asking' 'assure' 'at' 'ate' 'atmosphere' 'atrocious' 'attack'
 'attention' 'attentive' 'attitudes' 'authentic' 'average' 'avocado'
 'avoid' 'avoided' 'away']


In [16]:
# задаем список стоп-слов
stop_wrds = ['be', 'is', 'are', 'the', 'a',
             'an', 'on', 'of', 'and', 'in']

# задаем конвейер
pipe = Pipeline([
    ('vectorizer', CountVectorizer(stop_words=stop_wrds)), 
    ('logreg', LogisticRegression(solver='liblinear'))
])

# задаем сетку гиперпараметров
param_grid = {
    'vectorizer__ngram_range': [(1, 1), (1, 2), (2, 2)],
    'logreg__C': [.01, .1, .5, 1, 5, 10, 100, 150]
}

# создаем экземпляр класса GridSearchCV, передав конвейер,
# сетку гиперпараметров и указав количество
# блоков перекрестной проверки
gs = GridSearchCV(pipe, 
                  param_grid, 
                  cv=10)
# выполняем поиск по всем значениям сетки
gs.fit(X_train, y_train)
# смотрим наилучшие значения гиперпараметров
print("Наилучшие значения гиперпараметров:\n{}".format(
    gs.best_params_))
# смотрим наилучшее значение правильности
print("Наилучшее значение правильности: {:.3f}".format(
    gs.best_score_))
# смотрим значение правильности на тестовой выборке
print("Значение правильности на тестовой выборке: {:.3f}".format(
    gs.score(X_test, y_test)))

Наилучшие значения гиперпараметров:
{'logreg__C': 100, 'vectorizer__ngram_range': (1, 1)}
Наилучшее значение правильности: 0.796
Значение правильности на тестовой выборке: 0.803


In [17]:
# импортируем классы TfidfVectorizer и TfidfTransformer
from sklearn.feature_extraction.text import (TfidfVectorizer, 
                                             TfidfTransformer)
# создаем экземпляр класса TfidfVectorizer
tdidfvectorizer = TfidfVectorizer(smooth_idf=True)
# создаем игрушечный корпус из двух 
# документов - предложений
toy_corpus = ['Этот автомобиль едет', 
              'Этот грузовик едет']
# выполним масштабирование tf-idf 
tfidf = tdidfvectorizer.fit_transform(toy_corpus)
tfidf.toarray()

array([[0.70490949, 0.        , 0.50154891, 0.50154891],
       [0.        , 0.70490949, 0.50154891, 0.50154891]])

In [18]:
# задаем новую сетку гиперпараметров
param_grid2 = {
    'vectorizer__ngram_range': [(1, 1), (1, 2), (2, 2)],
    'vectorizer__max_features': [2000, 2500, 3000, 3500, 4000, 
                                 4500, 5000, 6000, 7000],
    'logreg__C': [.01, .1, .5, 1, 5, 10, 
                  100, 150, 200, 300]
}

# задаем конвейер
pipe2 = Pipeline([
    ('vectorizer', CountVectorizer(stop_words=stop_wrds)), 
    ('tfidftransformer', TfidfTransformer()),
    ('logreg', LogisticRegression(solver='liblinear'))
])

# создаем экземпляр класса GridSearchCV
gs2 = GridSearchCV(pipe2, 
                   param_grid2, 
                   cv=10, 
                   return_train_score=False)
# выполняем поиск по всем значениям сетки
gs2.fit(X_train, y_train)

# смотрим наилучшие значения гиперпараметров
print("Наилучшие значения гиперпараметров:\n{}".format(
    gs2.best_params_))
# смотрим наилучшее значение правильности
print("Наилучшее значение правильности: {:.3f}".format(
    gs2.best_score_))
# смотрим значение правильности на тестовой выборке
print("Значение правильности на тестовой выборке: {:.3f}".format(
    gs2.score(X_test, y_test)))

Наилучшие значения гиперпараметров:
{'logreg__C': 5, 'vectorizer__max_features': 4000, 'vectorizer__ngram_range': (1, 2)}
Наилучшее значение правильности: 0.800
Значение правильности на тестовой выборке: 0.807


In [19]:
# импортируем библиотеку googletrans
import googletrans
from googletrans import Translator
translator = Translator()

In [20]:
# функция обратного перевода текста
def back_translation(text):
    ru_text = translator.translate(text, dest='ru', src='en').text
    en_text = translator.translate(ru_text, dest='en', src='ru').text
    
    return en_text

In [21]:
# выполняем обратный перевод для обучающего набора
X_train_back = X_train.apply(back_translation)

In [22]:
# смотрим размер исходного обучающего набора
# и переведенного обучающего набора
print(X_train.shape, X_train_back.shape)

(700,) (700,)


In [23]:
# смотрим первые 5 текстов исходного
# обучающего набора
print(X_train.head())
print("")
# смотрим первые 5 текстов переведенного 
# обучающего набора
print(X_train_back.head())

284    I would definitely recommend the wings as well...
5         Now I am getting angry and I want my damn pho.
368          The staff are great, the ambiance is great.
670                                        dont go here.
413     I can assure you that you won't be disappointed.
Name: Review, dtype: object

284      I definitely recommend wings, as well as pizza.
5                 Now I am angry and want my damned FOR.
368    The staff is magnificent, the atmosphere is ma...
670                                      Do not go here.
413    I can assure you that you will not be disappoi...
Name: Review, dtype: object


In [24]:
# объединяем исходный и переведенный обучающие наборы
X_train_new = pd.concat([X_train, X_train_back])
y_train_new = pd.concat([y_train, y_train])

In [25]:
# избавляемся от записей с одинаковым текстом обзора
y_train_new = y_train_new[~X_train_new.duplicated()]
X_train_new.drop_duplicates(inplace=True)
print(y_train_new.shape, X_train_new.shape)

(1349,) (1349,)


In [26]:
# проверяем, нет ли в тестовом наборе текстов, 
# которые есть в новом обучающем наборе
np.intersect1d(X_train_new.values, X_test.values)

array(['The food was terrible.'], dtype=object)

In [27]:
# конкатенируем тестовые массив признаков и массив меток
test_data = pd.concat([X_test, y_test], axis=1)
# формируем тестовый набор так, чтобы в нем не было текстов,
# имеющихся в новом обучающем наборе
test_data = test_data[~test_data['Review'].isin(X_train_new)]
# формируем новый тестовый массив признаков
X_test = test_data['Review']
# формируем новый тестовый массив меток
y_test = test_data['Liked']

In [28]:
# выполняем поиск по всем значениям сетки
gs2.fit(X_train_new, y_train_new)
print("Наилучшие значения гиперпараметров:\n{}".format(
    gs2.best_params_))
print("Наилучшее значение правильности: {:.3f}".format(
    gs2.best_score_))
print("Значение правильности на тестовой выборке: {:.3f}".format(
    gs2.score(X_test, y_test)))

Наилучшие значения гиперпараметров:
{'logreg__C': 150, 'vectorizer__max_features': 7000, 'vectorizer__ngram_range': (1, 2)}
Наилучшее значение правильности: 0.961
Значение правильности на тестовой выборке: 0.833
